In [4]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
to_run = True
if to_run:
    os.chdir('..')
    to_run = False
print(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/workspaces/codespaces-jupyter
/workspaces


# Option 1 & 2: example input

In [5]:
import pandas as pd
import pm4py

from pm4py.objects.dcr.hierarchical.obj import HierarchicalDcrGraph
graph = HierarchicalDcrGraph()

events = [
    "A", "B", "C", "D", "E", "F"
]

groups = [    
    "N1", "N2", "N3"
]

for event in events:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event
    graph.marking.included.add(event)


for event in groups:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event
    #graph.nestedgroups_map[event] = event

    # graph.nestedgroups["B"] = {"B"}

graph.nestedgroups["N1"] =  {"B", "N2"} 
graph.nestedgroups["N2"] =  {"C", "D"} 
graph.nestedgroups["N3"] =  {"E", "F"} 

graph.marking.included.remove("B")
graph.marking.pending.add("D")


graph.conditions["N1"] = {"A"}
graph.conditions["N3"] = {"N1"}

graph.responses["N2"] = {"N3"}

graph.includes["N3"] = {"B"}

graph.excludes["C"] = {"N2"}

#ADDED
graph.__init__(graph.obj_to_template())
print(f"includes {graph.nestedgroups_map}")

includes {'N2': 'N1', 'B': 'N1', 'D': 'N2', 'C': 'N2', 'E': 'N3', 'F': 'N3'}


# Option 1 example output

In [6]:
import pandas as pd
from io import StringIO
import pm4py
from pm4py.objects.dcr.hierarchical import semantics as Hiearchical_Semantics   # Should be HierarchicalDcrSemantics instead!

semantics = Hiearchical_Semantics.HiearchicalSemantics()

errors = 0
def flat(e_prime_list:set):
            f = set()
            for e_prime in e_prime_list:
                if e_prime in graph.nestedgroups.keys():
                    for event in graph.nestedgroups[e_prime]:
                        f = f | flat({event})
                else:
                    f = f | {e_prime}
            return f

def condition_violated(e):
            for e_prime in graph.conditions[e]:
                if e_prime in graph.nestedgroups.keys():
                    c= flat(graph.nestedgroups[e_prime])
                    if len(c.intersection(graph.marking.included)) == 0:
                        return 1
                    elif len(c.intersection(graph.marking.included.intersection(
                    graph.marking.executed))) == 0:
                        return 1
                else:
                    if len({e_prime}.intersection(graph.marking.included.difference(
                    graph.marking.executed))) > 0:
                        return 1
            return 0

expected_enabled = ["A"]
print(f"\nexecuted {graph.marking.executed}")
print(f"pending {graph.marking.pending}")
print(f"included {graph.marking.included}")
print(f"enabled: {expected_enabled} vs {semantics.enabled(graph)}")
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 #raise Exception("Graph should NOT be accepting")        


semantics.execute(graph, "A")
expected_enabled = ["A", "C", "D"]
print(f"\nexecuted {graph.marking.executed}")
print(f"pending {graph.marking.pending}")
print(f"included {graph.marking.included}")
print(f"enabled: {expected_enabled} vs {semantics.enabled(graph)}")
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 #raise Exception("Graph should NOT be accepting")                

semantics.execute(graph, "C")
expected_enabled = ["A", "E", "F"]
print(f"\nexecuted {graph.marking.executed}")
print(f"pending {graph.marking.pending}")
print(f"included {graph.marking.included}")
print(f"enabled: {expected_enabled} vs {semantics.enabled(graph)}")
for e in set(graph.conditions.keys()).intersection(graph.nestedgroups.keys()):
            if(condition_violated(e)):
                print(f"discard{e}")
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 #raise Exception("Graph should NOT be accepting")                


semantics.execute(graph, "F")
expected_enabled = ["A", "B"]
print(f"\nexecuted {graph.marking.executed}")
print(f"pending {graph.marking.pending}")
print(f"included {graph.marking.included}")
print(f"enabled: {expected_enabled} vs {semantics.enabled(graph)}")
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 #raise Exception("Graph should NOT be accepting")   
        
semantics.execute(graph, "B")
expected_enabled = ["A", "B", "E", "F"]
print(f"\nexecuted {graph.marking.executed}")
print(f"pending {graph.marking.pending}")
print(f"included {graph.marking.included}")
print(f"enabled: {expected_enabled} vs {semantics.enabled(graph)}")
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should NOT be enabled")
if semantics.is_accepting(graph):
        errors+=1 #raise Exception("Graph should NOT be accepting")    

semantics.execute(graph, "E")
expected_enabled = ["A", "B", "E", "F"]
print(f"\nexecuted {graph.marking.executed}")
print(f"pending {graph.marking.pending}")
print(f"included {graph.marking.included}")
print(f"enabled: {expected_enabled} vs {semantics.enabled(graph)}")
for e in events:
    if e in expected_enabled:
            if not semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should be enabled")
    if not (e in expected_enabled):
        if semantics.is_enabled(e, graph):
                errors+=1 #raise Exception("Event should NOT be enabled")
if not semantics.is_accepting(graph):
        errors+=1 #raise Exception("Graph should be accepting") 

errors # This should be 0



executed set()
pending {'D'}
included {'E', 'F', 'A', 'D', 'C'}
enabled: ['A'] vs {'A'}

executed {'A'}
pending {'D'}
included {'E', 'F', 'A', 'D', 'C'}
enabled: ['A', 'C', 'D'] vs {'A', 'D', 'C'}

executed {'A', 'C'}
pending {'E', 'F', 'D'}
included {'E', 'F', 'A'}
enabled: ['A', 'E', 'F'] vs {'E', 'F', 'A'}
discardN3

executed {'A', 'F', 'C'}
pending {'E', 'D'}
included {'E', 'F', 'B', 'A'}
enabled: ['A', 'B'] vs {'A', 'B'}

executed {'A', 'F', 'B', 'C'}
pending {'E', 'D'}
included {'E', 'F', 'B', 'A'}
enabled: ['A', 'B', 'E', 'F'] vs {'E', 'F', 'A', 'B'}

executed {'E', 'F', 'B', 'A', 'C'}
pending {'D'}
included {'E', 'F', 'B', 'A'}
enabled: ['A', 'B', 'E', 'F'] vs {'E', 'F', 'A', 'B'}


0